In [1]:
model_name = "cnn_mixup_cutout_SGD"

In [2]:
import sys
import os
# Add project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)
from utils.paths import MODELS_DIR, DATA_DIR

In [7]:
# CIFAR10-Torch-Classifier and config.py
from core.cifar10_classifier import CIFAR10Classifier
import torch
import pandas as pd

In [4]:
config_path = os.path.join(MODELS_DIR, model_name,  f"{model_name}_config.json")
model_path = os.path.join(MODELS_DIR, model_name,  f"{model_name}_best_model.pth")

assert os.path.exists(config_path), f"Config not found at {config_path}"
assert os.path.exists(model_path), f"Model not found at {model_path}"

model = CIFAR10Classifier.load_model(
    model_name=model_name,
    config_path=config_path,
    model_path=model_path
)

model.summary()
mean, std = torch.tensor(model.mean), torch.tensor(model.std)

In [5]:
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import os

class KaggleCIFAR10Dataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_paths = sorted([
            os.path.join(image_dir, fname)
            for fname in os.listdir(image_dir)
            if fname.endswith(".png")
        ], key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, os.path.basename(img_path)


In [6]:
kaggle_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [17]:
from tqdm import tqdm

def generate_kaggle_submission(model, dataloader, class_names, output_path="submission.csv", device="cuda"):
    model.model.eval()
    predictions = []

    with torch.no_grad():
        for images, image_ids in tqdm(dataloader, desc="📤 Generating predictions", unit="batch"):
            images = images.to(device)
            outputs = model.model(images)
            _, predicted = torch.max(outputs, 1)

            for img_id, label in zip(image_ids, predicted.cpu().numpy()):
                predictions.append((img_id, class_names[label]))

    df = pd.DataFrame(predictions, columns=["Id", "Label"])
    df.to_csv(output_path, index=False)
    print(f"\n✅ Saved submission to {output_path}")


In [ ]:
# prepare dataset and loader
image_path = os.path.join(DATA_DIR, "Kaggle", "test")
dataset = KaggleCIFAR10Dataset(
    image_dir=image_path,
    transform=kaggle_transform
)
loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=False)

# generate predictions
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
generate_kaggle_submission(model, loader, class_names)


📤 Generating predictions:   9%|▉         | 206/2344 [01:52<41:08,  1.15s/batch]